# Creación de otros componentes que forman parte del modelo

La gran parte de los componentes que forman parte de la arquitectura de la red neuronal artificial pueden sustituirse por componentes personalizados de la misma forma

In [ ]:
def my_softplus(z): # Función de ACTIVACIÓN personalizada
    return tf.math.log(tf.exp(z) + 1.0)
def my_glorot_initializer(shape, dtype=tf.float32): # Función de INCIALIZACION personalizada
    stddev = tf.sqrt(2. / (shape[0] + shape[1]))
    return tf.random.normal(shape, stddev=stddev, dtype=dtype)
def my_l1_regularizer(weights): # Función de REGULARIZACION personalizada
    return tf.reduce_sum(tf.abs(0.01 * weights))
# RESTRICCION personalizada aplicada al valor de los parámetros de una capa
def my_positive_weights(weights):
    return tf.where(weights < 0., tf.zeros_like(weights), weights)

In [ ]:
from tensorflow import keras
# Definición de una capa de la red neuronal con los componentes personalizados
layer = keras.layers.Dense(1, activation=my_softplus,
                           kernel_initializer=my_glorot_initializer,
                           kernel_regularizer=my_l1_regularizer,
                           kernel_constraint=my_positive_weights)

In [ ]:
network = models.Sequential()
network.add(layers.Dense(30, activation='relu', input_shape=X_train.shape[1:]))
network.add(layers.Dense(10, activation='relu'))
network.add(layers.Dense(1, activation=my_softplus,
                         kernel_initializer=my_glorot_initializer,
                         kernel_regularizer=my_l1_regularizer,
                         kernel_constraint=my_positive_weights))

En el caso de que la funcion contenga parametros que queremos almacenar/guardar junto al modelo, la mejor forma es crear una subclase de _tf.keras.*_

In [ ]:
# Función de regularización personalizada
class MyL1Regularizer(tf.keras.regularizers.Regularizer):
    def __init__(self, factor):
        self.factor = factor
    def __call__(self, weights):
        return tf.reduce_sum(tf.abs(self.factor * weights))
    def get_config(self):
        return {"factor": self.factor}

In [ ]:
network = models.Sequential()
network.add(layers.Dense(30, activation='relu', input_shape=X_train.shape[1:]))
network.add(layers.Dense(10, activation='relu'))
network.add(layers.Dense(1, activation=my_softplus,
                         kernel_initializer=my_glorot_initializer,
                         kernel_regularizer=MyL1Regularizer(0.01),
                         kernel_constraint=my_positive_weights))

In [ ]:
network.save("modelo_personalizado.h5")

In [ ]:
# Cargamos el modelo previamente almacenado en disco
model = keras.models.load_model(
    "modelo_personalizado.h5",
    custom_objects={
       "MyL1Regularizer": MyL1Regularizer,
       "my_positive_weights": my_positive_weights,
       "my_glorot_initializer": my_glorot_initializer,
       "my_softplus": my_softplus,
    })

# Metricas personalizadas

Se utilizan exactamente igual que todos los componentes vistos anteriormente

In [ ]:
network.compile(loss="mse", optimizer="sgd", metrics=[create_huber(2.0)])